# Logistic Regression

Logistic regression predicts binary (yes/no) events. For example, we may want to predict if someone will arrive at work on time, or if a person shopping will buy a product.

This exercise will demonstrate simple logistic regression: predicting an outcome from only one feature.

## Step 1 - Import NuGet packages

Necessary NuGet packages can easily be imported to use it in a Jupyter Notebook using the following code. In this case we will need Microsot.ML and Xplot.Plotly for the graphics.

In [ ]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML,1.4"
    
//Install XPlot package
#r "nuget:XPlot.Plotly,2.0.0"

using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using XPlot.Plotly;

## Step 2 - Storing the data

First of all, we 're going to define some basic auxiliary class and functions.

In [ ]:
public static double Sigmoid(double x)
{
    return 1 / (1 + Math.Exp(-x));
}

In [ ]:
public class PlotChartPoint
{
    public double X { get; set; }

    public double Y { get; set; }
}

In [ ]:
public static IEnumerable<PlotChartPoint> GetAvgChartPointsFromData(IEnumerable<double> x, IEnumerable<double> y)
{
    var points = new List<PlotChartPoint>();
    for (int i = 0; i < x.Count(); i++)
    {
        points.Add(new PlotChartPoint()
        {
            X = x.ElementAt(i),
            Y = y.ElementAt(i)
        });
    }

    return points;
}

You will also need to create a class suited to store the information you have. Once this is done, you can load structured information into this class and verify  it by showing the schema and some rows of the DataView.

In [ ]:
public class FootballInput
{
    [LoadColumn(0)]
    [ColumnName("AverageGoalsPerMatch")]
    public float AverageGoalsPerMatch { get; set; }

    [LoadColumn(1)]
    [ColumnName("WonCompetition")]
    public bool WonCompetition { get; set; }
}

public class FootballOutput
{
    [ColumnName("Score")]
    public float WonCompetition { get; set; }
}

In [ ]:
string TrainDataPath = "./Data/football-data.txt";

MLContext mlContext = new MLContext(seed:0);
IDataView dataView = mlContext.Data.LoadFromTextFile<FootballInput>(path: TrainDataPath, hasHeader: true, separatorChar:'\t');

display(dataView.Schema);

In the cell below replace the text `<printDataHere>` with display(fewRows); and then press Run in the toolbar above (or press Shift+Enter).

In [ ]:
public static List<FootballInput> Head(MLContext mlContext, IDataView dataView, int numberOfRows = 4)
{
    string msg = string.Format("DataView: Showing {0} rows with the columns", numberOfRows.ToString());
    display(msg);
          
    var rows = mlContext.Data.CreateEnumerable<FootballInput>(dataView, reuseRowObject: false)
                    .Take(numberOfRows)
                    .ToList();
    
    return rows;
}

display(h4("Showing a few rows from training DataView:"));

var fewRows = Head(mlContext, dataView, 10);

/*
 REPLACE <PrintDataHere> WITH display(fewRows);
*/
<PrintDataHere>
//

## Step 3 - Visualize Data

Let's graph the data so we have a better idea of what's going on here. Complete the exercise below to make an x-y scatter plot.

The following chart will show us the distribution of victories according to the number of average goals.

In [ ]:
int numberOfRows = 100;
float[] goals = dataView.GetColumn<float>("AverageGoalsPerMatch").Take(numberOfRows).ToArray();
bool[] won = dataView.GetColumn<bool>("WonCompetition").Take(numberOfRows).ToArray();
float[] won_float = won.Select(x => Convert.ToSingle(x)).ToArray();

Then, we set the chart and the layout options and display the result.

In the cell below replace the text `<printDataHere>` with display(chart);

In [ ]:
var chart = Chart.Plot(
    new Graph.Scatter()
    {
        x = goals,
        y = won_float,
        mode = "markers",
        marker = new Graph.Marker()
        {
            color = won_float,
            colorscale = "Jet"
        }
    }
);

var layout = new Layout.Layout(){title="Average goals vs competition won"};
chart.WithLayout(layout);
chart.WithXTitle("Average goals per Match");
chart.WithYTitle("Competition won");
chart.WithLabels(new[]{"Goals"});
chart.Width = 700;
chart.Height = 500;
chart.WithLegend(true);

/*
 REPLACE <PrintDataHere> WITH display(chart);
*/
<PrintDataHere>
//

## Step 4 -Building a model and running a prediction

How can we predict whether the team will win this season? Let's apply AI to this problem, by making a logisitic regression model using this data and then graph it. This will tell us whether we will likely win this season.

The first thing you will need to do is to create a pipeline. In this case we are creating a pipeline using the LogisticRegression algorithm, a well-known statistical method for determining the contribution of multiple factors to a pair of outcomes

In [ ]:
var pipeline =
    mlContext.Transforms.Concatenate("Features", "AverageGoalsPerMatch")
    .Append(mlContext.BinaryClassification.Trainers.LbfgsLogisticRegression("WonCompetition"));

The next step is to train our model by passing our training data to the method Fit.

In [ ]:
// Train the model
var model = pipeline.Fit(dataView);

Use the model to work out the loss.

In [ ]:
// View training stats
var linearModel = model.LastTransformer.Model.SubModel as LinearBinaryModelParameters;

// This works out the loss
var coefficient = linearModel.Weights.FirstOrDefault();
var intercept = linearModel.Bias;
var step = 3 / (double)300;
var testX = Enumerable.Range((int)0, 300).Select(v => (v * step) + 0).ToList();
var loss = new List<double>();
foreach (var x in testX)
{
    loss.Add(Sigmoid(x * coefficient + intercept));
}

// Get an array of the average data points
var lossPoints = GetAvgChartPointsFromData(testX, loss);
var X = lossPoints.Select(PlotChartPoint => PlotChartPoint.X);
var Y = lossPoints.Select(PlotChartPoint => PlotChartPoint.Y);

Now we can see graphically the regression line obtained by the  model, which will show us the probability of winning a competition according to the average goals.

In the cell below replace the text `<printDataHere>` with display(chart);

In [ ]:
// Define grapgh for the line 

var lineGraph= new Graph.Scatter()
    {
        x = X.ToList(),
        y = Y.ToList(),
        mode = "lines",
        line = new Graph.Line(){color = "black"}
    };

var pointsGraph = new Graph.Scatter()
    {
        x = goals,
        y = won_float,
        mode = "markers",
        marker = new Graph.Marker()
        {
            color = won_float,
            colorscale = "Jet"
        }
    };

var regressionLine = Chart.Plot(new []{pointsGraph,lineGraph});

var layout = new Layout.Layout(){title="Competition Win Likelihood"};
chart.WithLayout(layout);
chart.WithXTitle("Average goals per Match");
chart.WithYTitle("Competition won");
chart.WithLabels(new[]{"Goals","Win Likelihood"});
chart.WithLegend(true);
chart.Width = 700;
chart.Height = 500;

/*
 REPLACE <PrintDataHere> WITH display(chart);
*/
<regressionLine>
//

## Step 5 - Try a single prediction

We can read the model above like so:

<ul>
<li>Take the average number of goals per match for the current year. Let's say it is 2.5.
<li>Find 2.5 on the x-axis.
<li>What value (on the y axis) does the line have at x=2.5?
<li>If this value is above 0.5, then the model thinks our team will win this year. If it is less than 0.5, it thinks our team will lose.
<li>Because this line is just a mathematical function (equation) we don't have to do this visually.
</ul>

In the exercise below, <b>choose the number of goals you want to evaluate</b>.

The code will calculate the probability that our team will win with your chosen number of goals in the match.

In the cell below replace the text `<GoalsPerMatch>` with 2.422870462f

In [ ]:
// Use the trained model for one-time prediction
var predictionEngine = mlContext.Model.CreatePredictionEngine<FootballInput, FootballOutput>(model);

// Features to include in the prediction
/*
 REPLACE <GoalsPerMatch> WITH 2.422870462f
*/
var goalsPerMatch =  <GoalsPerMatch>;
//    

var prediction = predictionEngine.Predict(new FootballInput
{
    AverageGoalsPerMatch = goalsPerMatch
});

Console.WriteLine($"*************************************");
Console.WriteLine($"Probability of winning this year: { prediction.WonCompetition * 100 }%");
Console.WriteLine($"*************************************");